# Script to calibrate the Furka PMOS and push calibration
### Purpose: Calibtraion of dispersed energy axis from pixel to eV for PMOS
Contact christopher.arrell@psi.ch

In [5]:
%load_ext autoreload
import epics as ep
import numpy as np
from PMOS_tools import *
import matplotlib.pyplot as plt
from collections import deque
from scipy.optimize import curve_fit
%matplotlib inline
from IPython.display import clear_output, display
import h5py as h5
from scipy.optimize import curve_fit
import seaborn as sns
from lmfit.models import ConstantModel, GaussianModel, VoigtModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Move in PMOS screen

In [7]:
%autoreload
set_PMOS_screen_y(26)

it worked


## Setup and functions setup

In [ ]:
PMOS_e_axis = PMOS_e_axis_PV.get()

   

def PMOS_focus_scan(Focus_range, NumShots):
        
    Spectrum_scan = []
    e_axis_scan = []
    
    for focus_val in Focus_range:
        set_PMOS_focus(focus_val)  
           
        Spectrum_this_energy = deque(maxlen = NumShots)
        def on_value_change(value=None,pv = None, **kwargs):
            Spectrum_this_energy.append(value)

            if len(Spectrum_this_energy) == NumShots:
                pv.clear_callbacks()
                
        PMOS_spectrum_PV.add_callback(callback=on_value_change, pv =PMOS_spectrum_PV)
        while len(Spectrum_this_energy) < NumShots:
            sleep(1)
        Spectrum_scan.append(np.array(Spectrum_this_energy))
        e_axis_scan.append(PMOS_e_axis_PV.get())
    return(np.array(e_axis_scan),np.array(Spectrum_scan))

def MONO_r2_scan(r2_range, NumShots):
        
    Spectrum_scan = []
    e_axis_scan = []
    
    for r2 in r2_range:
        set_MONO_r2(r2)  
           
        Spectrum_this_energy = deque(maxlen = NumShots)
        def on_value_change(value=None,pv = None, **kwargs):
            Spectrum_this_energy.append(value)

            if len(Spectrum_this_energy) == NumShots:
                pv.clear_callbacks()
                
        PMOS_spectrum_PV.add_callback(callback=on_value_change, pv =PMOS_spectrum_PV)
        while len(Spectrum_this_energy) < NumShots:
            sleep(1)
        Spectrum_scan.append(np.array(Spectrum_this_energy))
        e_axis_scan.append(PMOS_e_axis_PV.get())
    return(np.array(e_axis_scan),np.array(Spectrum_scan))

def PMOS_screen_y_scan(y_range, NumShots):
        
    Spectrum_scan = []
    e_axis_scan = []
    
    for y in y_range:
        set_PMOS_screen_y(y)  
           
        Spectrum_this_energy = deque(maxlen = NumShots)
        def on_value_change(value=None,pv = None, **kwargs):
            Spectrum_this_energy.append(value)

            if len(Spectrum_this_energy) == NumShots:
                pv.clear_callbacks()
                
        PMOS_spectrum_PV.add_callback(callback=on_value_change, pv =PMOS_spectrum_PV)
        while len(Spectrum_this_energy) < NumShots:
            sleep(1)
        Spectrum_scan.append(np.array(Spectrum_this_energy))
        e_axis_scan.append(PMOS_e_axis_PV.get())
    return(np.array(e_axis_scan),np.array(Spectrum_scan))

def gaus(x,a,x0,sigma,offset):
    return offset +a*np.exp(-(x-x0)**2/(2*sigma**2))

def Spectra_bin(Scan_e_axis, e_axis_full, spectra):
    digi = np.digitize(Scan_e_axis, e_axis_full)   
    Spec_binned = []
    for i in range(0,len(e_axis_full)):
        ind = digi==i+1
        Spec_binned.append(spectra[ind].mean(axis=0))
    return np.asarray(Spec_binned)

def gauss(x, H, A, x0, sigma):
    return H + A * np.exp(-(x - x0) ** 2 / (2 * sigma ** 2))

def bimodal(x, H_1, A_1, x0_1, sigma_1,H_2, A_2, x0_2, sigma_2):
    return gauss(x, H_1, A_1, x0_1, sigma_1)+gauss(x, H_2, A_2, x0_2, sigma_2)

def gauss_fit(x, y):
    mean = sum(x * y) / sum(y)
    sigma = np.sqrt(sum(y * (x - mean) ** 2) / sum(y))
    popt, pcov = curve_fit(gauss, x, y, p0=[min(y), max(y), mean, sigma])
    return popt

def bimodal_fit(x, y):
    mean = sum(x * y) / sum(y)
    sigma = np.sqrt(sum(y * (x - mean) ** 2) / sum(y))
    popt, pcov = curve_fit(bimodal, x, y, p0=[min(y), max(y), mean, sigma,min(y), max(y), mean, sigma])
    return popt

def auto_corr(e_axis, Spec):
    correlations = np.zeros_like(Spec)
    correlations = []
    
    for i in range(0,Spec.shape[0]):
        test = np.correlate(Spec[i,:].astype('float'),Spec[i,:].astype('float'), mode='same')
#         test = np.correlate(Spec[i,:],Spec[i,:], mode='same')
        
        correlations.append(test)
    lags = e_axis - e_axis[int(e_axis.size /2)]
    correlations = np.asarray(correlations)
    return lags, correlations.mean(axis=0)

## User inputs - define focus range to scan 

Scan range

In [ ]:
Focus_from = 0.4
Focus_to = 0.6
steps = 20
NumShots= 10
Focus_range = np.linspace(Focus_from, Focus_to, steps)

In [ ]:
r2_from = 40
r2_to = 50
steps = 10
NumShots= 100
r2_range = np.linspace(r2_from, r2_to, steps)

In [ ]:
y_from = 26
y_to = 36
steps = 10
NumShots= 10
y_range = np.linspace(y_from, y_to, steps)

Only run the follow cell if you want to save in a single energy position



## Scan to calibrate PMOS

In [ ]:
set_PMOS_screen_y(y_range[0])
Scan_e_axis, Scan_spec = PMOS_screen_y_scan(y_range,NumShots)

In [ ]:
np.argmax(Scan_spec[0,:,:].mean(axis=0))

In [ ]:
max_pixel = []
for i in range(0, Scan_spec.shape[0]):
    max_pixel.append(np.argmax(Scan_spec[i,:,:].mean(axis=0)))
    

In [ ]:
plt.figure()
plt.plot(y_range, max_pixel)

In [ ]:
grad = np.abs((max_pixel[0]-max_pixel[-1])/(y_range[0]-y_range[-1]))

In [ ]:
fov_mm = Scan_e_axis[0,-1]/grad

In [ ]:
energy_window =0.729*fov_mm

In [ ]:
central_engery = 532.5 #eV
energy_0 = central_engery-energy_window/2
energy_1 = central_engery+energy_window/2

In [ ]:
e_axis = np.linspace(energy_0, energy_1,Scan_e_axis.shape[1])

In [ ]:
e_axis[0]

In [ ]:
ep.caput('SATOP31-PMOS132-2D:SPECTRUM_X',e_axis)

In [ ]:
mean_cor = []
for i in range(0, Scan_spec.shape[0]):
    lags, this_cor = auto_corr(Scan_e_axis[0,:], Scan_spec[i,:,:]-50000)
    mean_cor.append(this_cor)
mean_cor = np.asarray(mean_cor)

In [ ]:
sns.set_style("darkgrid")
sns.set_context("talk")
plt.figure(figsize=[20,10])
plt.subplot(121)
plt.title('Mono R2 scan \n auto correlation')
plt.pcolormesh(lags,Focus_range,mean_cor)
plt.xlabel('lags')
plt.ylabel('Lens voltage')
plt.subplot(122)
index = int(len(Focus_range)/2)
plt.plot(lags, mean_cor[index,:], label = "Len_voltage = %.2f"%Focus_range[index])
plt.legend()
plt.xlabel('lags')

In [ ]:
plt.figure(figsize=[15,10])
plt.title('PMOS spectra centred at \n 400 [eV]')
plt.plot(Scan_spec[0,1,:])
# plt.plot(Scan_spec[0,2,:])
plt.savefig('Some_spec', dpi = 300)

In [ ]:
Scan_spec.shape

In [ ]:
# PMOS r2 scan

In [ ]:
set_MONO_r2(r2_range[0])
sleep(1)
Scan_e_axis, Scan_spec = MONO_r2_scan(r2_range,NumShots)

In [ ]:
Scan_spec_bkg_rm = np.zeros_like(Scan_spec)

In [ ]:
for i in range(0,Scan_spec.shape[0]):
    for j in range(0,Scan_spec.shape[1]):
        Scan_spec_bkg_rm[i,j,:] = Scan_spec[i,j,:]-50000

In [ ]:
model = GaussianModel(prefix="p1_") + GaussianModel(prefix="p2_") + GaussianModel(prefix="p3_")

model.set_param_hint("p1_center", value=0, vary=False)
model.set_param_hint("p2_center", value=0, vary=False)
model.set_param_hint("p3_center", value=0, vary=False)

In [ ]:
peak_width = []
spectral_width = []
bkg_width = []
peak_err = []
spectral_err = []
bkg_err = []
mean_cor =[]
min_width = []
for i in range(0, Scan_spec.shape[0]):
    lags, this_cor = auto_corr(Scan_e_axis[0,:], Scan_spec[i,:,:]-50000)
#     this_cor = np.asarray(this_cor)
    this_cor/= np.max(this_cor)
    lags = lags.astype("float64")
    params = model.make_params()
    result = model.fit(this_cor, params, x=lags, model ='least_squares')

    mean_cor.append(this_cor)
    peak_width.append(result.values['p1_sigma'])
    spectral_width.append(result.values['p2_sigma'])
    bkg_width.append(result.values['p3_sigma'])
    min_width.append(np.min(result.values['p1_sigma'] +result.values['p2_sigma']+result.values['p3_sigma']))

    peak_err.append(result.params['p1_sigma'].stderr)
    spectral_err.append(result.params['p2_sigma'].stderr)
    bkg_err.append(result.params['p3_sigma'].stderr)
mean_cor = np.asarray(mean_cor)

In [16]:
e_axis = np.arange(0,1456)
ep.caput('SATOP31-PMOS132-2D:SPECTRUM_X', e_axis)

1

In [14]:
1968-513

1455

In [ ]:
plt.figure()
plt.plot(lags, mean_cor[-1,:], label = 'Auto correlation')
plt.plot(lags, result.best_fit, label ='fit')
plt.legend()

In [ ]:
fits = np.zeros(shape = [3,len(r2_range)])
fits[0,:] = peak_width
fits[1,:] = spectral_width
fits[2,:] = bkg_width

In [ ]:
fits.shape

In [ ]:
min_width = []
for i in range(0, fits.shape[1]):
    min_width.append(np.min(fits[:,i]))

In [ ]:
min_width

In [ ]:
plt.figure()
plt.plot(r2_range, min_width)
plt.xlabel('r2')
plt.ylabel('sigma smallest fit')
# plt.ylim([0, 200])

In [ ]:
sns.set_style("darkgrid")
sns.set_context("talk")
plt.figure(figsize=[20,10])
plt.subplot(121)
plt.title('Mono R2 scan \n auto correlation')
plt.pcolormesh(lags,r2_range,mean_cor)
plt.xlabel('lags')
plt.ylabel('r2')
plt.subplot(122)
index = int(len(mean_cor)/2)
plt.plot(lags, mean_cor[-1,:], label = "R2 = %.2f"%r2_range[index])
plt.plot(lags, bimodal(lags, *bpopt))
plt.legend()
plt.xlabel('lags')

In [ ]:
sns.set_style("darkgrid")
sns.set_context("talk")
plt.figure(figsize=[10,10])
plt.title('Mono R2 scan')
plt.pcolormesh(Scan_e_axis.mean(axis=0),r2_range,Scan_spec.mean(axis=1))
plt.xlabel('Pixel')
plt.ylabel('r2')

In [ ]:
plt.figure()
plt.pcolormesh(Focus_range, Scan_e_axis.mean(axis=0), Scan_spec.mean(axis=1),cmap='CMRmap')


## Save data

In [ ]:
folder = "/sf/photo/two_colour_test/"
name = 'energy_scan_100.h5'

In [ ]:
with h5.File(folder+name, 'w-') as fh:
    fh['Scan_e_axis'] = Scan_e_axis
    fh['Scan_spec'] = Scan_spec
    fh['Energy_range'] = Energy_range
    fh['e_axis_full'] = e_axis_full
    fh['Bin_spec'] = Bin_spec
    fh['Spec_range'] = Spec_range    